# $SO(3)$ Pfaffian method

## tenpy Hamiltonian

Write the tenpy Hamiltonians with a single Cartan SubAlg as a $U(1)$ good quantum number. The first part is the same as lines in /mpomps/so3/cc.ipynb. 

In [3]:
import numpy as np
import numpy.linalg as LA

from tenpy.models.model import CouplingModel, MPOModel
from tenpy.networks.site import SpinSite, FermionSite, Site
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS
from tenpy.networks.mpo import MPO
import tenpy.linalg.np_conserved as npc
from tenpy.linalg.charges import LegCharge, ChargeInfo
from tenpy.algorithms import dmrg
from tenpy.tools.params import asConfig

In [4]:
def get_so3_opr_list_new():
    L1 = np.array([[-1, 0, 0], [0, 0, 0], [0, 0, 1]])
    L2 = (1j/np.sqrt(2))*np.array([[0, -1, 0], [1, 0, -1], [0, 1, 0]])
    L3 = -(1/np.sqrt(2))*np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
    
    Loprs = [L1, L2, L3]
    Lhatoprs = [
        np.array([
            [-1, 0, 0],
            [0, 0, 0],
            [0, 0, 1]
        ]),
        np.array([
            [0, 1, 0],
            [0, 0, 1],
            [0, 0, 0]
        ]),
        np.array([
            [0, 0, 0],
            [1, 0, 0],
            [0, 1, 0]
        ]),
        np.array([
            [1, 0, 0],
            [0, 0, 0],
            [0, 0, 1]
        ]),
        np.array([
            [0, 1, 0],
            [0, 0, 0],
            [0, 0, 0]
        ]),
        np.array([
            [0, 0, 0],
            [0, 0, 0],
            [0, 1, 0]
        ]),
        np.array([
            [0, 0, 1],
            [0, 0, 0],
            [0, 0, 0]
        ]),
        np.array([
            [0, 0, 0],
            [0, 0, 0],
            [1, 0, 0]
        ]),
        np.array([
            [0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]
        ]),
    ]

    coe_list = []

    for a in range(len(Loprs)):
        for b in range(len(Loprs)):
            LiLi = Loprs[a] @ Loprs[b]
            Amat = np.zeros((9, len(Lhatoprs)), dtype=complex)
            B = LiLi.reshape(-1,1)
            for l in range(len(Lhatoprs)):
                Amat[:,l] = Lhatoprs[l].reshape(-1,1)[:,0]
            pcoe = LA.solve(Amat, B)
            coe_list.append(pcoe)

    for i in range(len(coe_list)):
        coe_list[i] = coe_list[i].reshape(9)

    def pvec(a,b):
        return coe_list[len(Loprs)*a+b] #a,b=0,1,2,3
    
    cmn = np.zeros((9,9), dtype=complex)

    P = dict()
    for a in range(len(Loprs)):
        for b in range(len(Loprs)):
            P[(a,b)] = pvec(a,b)
    
    for m in range(9):
        for n in range(9):
            for a in range(len(Loprs)):
                for b in range(len(Loprs)):
                    cmn[m,n] += P[(a,b)][m] * P[(a,b)][n]

    coe_list_new = []
    for a in range(len(Loprs)):
        Li = Loprs[a]
        Amat = np.zeros((9, len(Lhatoprs)), dtype=complex)
        B = Li.reshape(-1,1)
        for l in range(len(Lhatoprs)):
            Amat[:,l] = Lhatoprs[l].reshape(-1,1)[:,0]
        qcoe = LA.solve(Amat, B)
        coe_list_new.append(qcoe)
    
    for i in range(len(coe_list_new)):
        coe_list_new[i] = coe_list_new[i].reshape(9)

    def qvec(a):
        return coe_list_new[a]
    
    dmn = np.zeros((9,9), dtype=complex)
    
    Q = dict()
    for a in range(len(Loprs)):
        Q[a] = qvec(a)

    for m in range(9):
        for n in range(9):
            for a in range(len(Loprs)):
                dmn[m,n] += Q[a][m] * Q[a][n]

    return Lhatoprs, cmn, dmn

In [5]:
class SO3Site(Site):
    def __init__(self, so4g, cons_N=None, cons_S=None):
        self.conserve = [cons_N, cons_S]
        self.cons_N = cons_N
        self.cons_S = cons_S
        self.so4g = so4g
        
        if self.cons_N is None and self.cons_S == 'U1':
            chinfo = npc.ChargeInfo([1], ['S'])
            leg = npc.LegCharge.from_qflat(chinfo, [[-1], [0], [1]])
        elif self.cons_N is None and self.cons_S is None:
            leg = npc.LegCharge.from_trivial(3)
        
        ops = dict()
        for i in range(len(self.so4g)):
            ops['L{}'.format(i)] = self.so4g[i]

        names = ['d', 'z', 'u']
        Site.__init__(self, leg, names, **ops)

    def __repr__(self):
        return "trivial site for 9 so3 generators"

In [6]:
class BBQJKSO3(CouplingModel):
    def __init__(self, model_params):
        print(model_params)
        model_params = asConfig(model_params, self.__class__.__name__)
        self.model_params = model_params
        self.Lx = model_params.get('Lx', 12)
        self.S = model_params.get('S', 1)
        self.bc = model_params.get('bc', 'periodic')
        self.J = model_params.get('J', 1)
        self.K = model_params.get('K', 1/3)
        self.verbose = model_params.get('verbose', 2)
        self.D = model_params.get('D', 200)
        self.sweeps = model_params.get('sweeps', 10)
        self.conn = model_params.get('conn', None)
        self.cons = model_params.get('cons', 'U1')
        
        self.so4_generators, self.c_mn, self.d_mn = get_so3_opr_list_new()

        site = SO3Site(self.so4_generators, cons_N=self.conn, cons_S=self.cons)
        self.sites = [site] * self.Lx
        self.lat = Chain(self.Lx, site, bc=self.bc)
        CouplingModel.__init__(self, self.lat, explicit_plus_hc=False)
        self.init_terms(model_params)
        H_MPO = self.calc_H_MPO()
        if model_params.get('sort_mpo_legs', False):
            H_MPO.sort_legcharges()
        MPOModel.__init__(self, self.lat, H_MPO)
        model_params.warn_unused()

    def init_terms(self, model_params):
        J = model_params.get("J", 1.)
        K = model_params.get('K', 1/3)
        for l in range(self.Lx):
            if l < self.Lx - 1:
                i0, i1 = l, (l+1)%self.Lx
            elif l == self.Lx-1 and self.bc == 'periodic':
                i0, i1 = 0, self.Lx-1
                print("periodic terms added")
            else:
                break
        
            for m in range(9):
                for n in range(9):
                    self.add_coupling_term(J*np.round(self.d_mn[m,n],6), i0, i1, "L"+str(m), "L"+str(n))
                    if K != 0:
                        self.add_coupling_term(K*np.round(self.c_mn[m,n],6), i0, i1, "L"+str(m), "L"+str(n))
    
    def run_dmrg(self, **kwargs):
        mixer      = kwargs.get('mixer', True)
        chi_max    = kwargs.get('chi_max', self.D)
        max_E_err  = kwargs.get('max_E_err', 1e-10)
        max_sweeps = kwargs.get('max_sweeps', self.sweeps)
        min_sweeps = kwargs.get('min_sweeps', min(4, max_sweeps) )
        dmrg_params = dict(mixer=mixer, 
                           trunc_params=dict(chi_max=chi_max),
                           max_E_err=max_E_err, 
                           max_sweeps=max_sweeps,
                           min_sweeps=min_sweeps,
                           verbose=2)

        init = kwargs.get('init', None)
        if init is None:
            N = self.lat.N_sites
            if N%6==0 and N>0:
                init = [0]*(N//3) + [1]*(N//3) + [2]*(N//3)
            else:
                raise("Check the system size must be integral multiple of 6")
            np.random.shuffle(init)
            psiinit = MPS.from_product_state(self.lat.mps_sites(), init)
            psiinit.norm = 1
            psiinit.canonical_form()
        elif isinstance(init, list):
            psiinit = MPS.from_product_state(self.lat.mps_sites(), init)            
        elif isinstance(init, MPS):
            psiinit = init
        else:
            print("wrong init")

        eng = dmrg.TwoSiteDMRGEngine(psiinit, self, dmrg_params)
        E, psidmrg = eng.run()
        print("Eng = ", E)
        self.psidmrg = psidmrg
        return psidmrg, E

In [7]:
import logging
logging.basicConfig(level=2)
for _ in ['parso.python.diff', 'parso.cache', 'parso.python.diff', 
            'parso.cache', 'matplotlib.font_manager', 'tenpy.tools.cache', 
            'tenpy.algorithms.mps_common', 'tenpy.linalg.lanczos', 'tenpy.tools.params']:
    logging.getLogger(_).disabled = True

## Pfaffian part

First we generate the BdG Hamiltonian of a Kitaev single chain. Then combine 3 copies of them by using the fermion swap gate method. 

In [8]:
from gaussianbdg import *
from copy import deepcopy

In [9]:
class singlechain(bdg):
    """
    The Bogoliubov-de Gennes form of a SINGLE Kitaev chain.
    """
    def __init__(self, chi, d, lamb, Nx, D, pbc):
        self.model = "Kitaev single chain_L{}_chi{}_d{}_lambda{}_D{}".format(Nx, round(chi,6), round(d,6), round(lamb,6), D)
        super().__init__(Nx=Nx, Ny=1, model=self.model, D=D)
        self.t, self.d = round(-chi, 6), round(d, 6)
        self.mu = lamb
        self.dtype = np.float64
        self.pbc = pbc
        self.Nlatt = Nx #the REAL site number
        
    def hamiltonian(self):
        N = self.Nlatt
        self.tmat = np.zeros((self.Nlatt, self.Nlatt), self.dtype)
        self.dmat = np.zeros((self.Nlatt, self.Nlatt), self.dtype)
        t, d = self.t, self.d
        mu = self.mu
        print("t=",self.t, "d=",self.d, "mu=",self.mu)
        for i in range(N):
            self.tmat[i, i] = mu/2 #why devided by 2 -> we will add the diagonal terms twice below
        for i in range(N-1):
            self.tmat[i, (i+1)%N] = t 
            self.dmat[i, (i+1)%N] = d 
        self.parity = 1
        if self.pbc:
            parity = - 1 + 2 * ( N % 2 )
            self.parity = parity
            self.tmat[N-1, 0] = t*parity 
            self.dmat[N-1, 0] = d*parity 
        self.tmat += self.tmat.T.conj() #here we doubled the diagonal elements
        self.dmat -= self.dmat.T
        
        self.ham = np.block([[self.tmat, self.dmat],[-self.dmat.conj(), -self.tmat.conj()]])
        self.eig_eng, self.eig_vec = bdgeig_zeromode(self.ham, ref=0.0)
        print("the eig energies", self.eig_eng)
        self.exact_EgsXY = -self.eig_eng[:self.Nlatt].sum()/2 + np.trace(self.tmat)/2 - self.Nlatt*mu/2
        print("the exact energy by em and trt", -self.eig_eng[:self.Nlatt].sum()/2 + np.trace(self.tmat)/2 - self.Nlatt*mu/2)
        self.u, self.v = m2uv(self.eig_vec)
        self.m = uv2m(self.u, self.v)
        print("check the hamiltonian diagonalization: HM=ME, M=[[u,v*];[v,u*]]")
        check_orthonormal(self.ham, self.m)
        self.covariance()

In [ ]:
class oneparton(Site):
    """
    Site of single parton, occupied or unoccupied, dimension=2
    """
    def __init__(self, flavor=None, cons_S=None, cons_N=None):
        self.cons_N = cons_N

        if cons_N == None and cons_S == 'U1':
            if flavor == 'x':
                chinfo = npc.ChargeInfo([1], ['S'])
                leg = npc.LegCharge.from_qflat(chinfo, [[0], [-1]])
            elif flavor == 'y':
                chinfo = npc.ChargeInfo([1], ['S'])
                leg = npc.LegCharge.from_qflat(chinfo, [[0], [0]])
            elif flavor == 'z':
                chinfo = npc.ChargeInfo([1], ['S'])
                leg = npc.LegCharge.from_qflat(chinfo, [[0], [1]])
            else:
                raise("flavor must be x, y, or z")
        elif cons_N == 'Z2' and cons_S == None:
            chinfo = npc.ChargeInfo([2], ['N'])
            leg = npc.LegCharge.from_qflat(chinfo, [[0], [1]])
        elif cons_N == None and cons_S == None:
            leg = npc.LegCharge.from_trivial(2)

        names = ['unoccupied', 'occupied']
        
        id2 = npc.diag([1,1], leg)
        ops = dict(id=id2)
        
        Site.__init__(self, leg, names, **ops)

def fermion_swap_gate_z2(legv, legp):
    #The charges must be set for every leg
    fswap = npc.zeros((legv, legp, legv.conj(),legp.conj()), labels=('fvR','sp','fvL','sp*'))
    for _v in range(legv.block_number):
        cv = legv.charges[_v]*legv.qconj
        pv = 1 - 2*(cv[0]%2)
        qv = legv.get_qindex_of_charges(cv)
        sv = legv.get_slice(qv)
        #print('pv', pv, 'cv', cv, 'qv', qv,'sv', sv)
        for _p in range(legp.block_number):
            cp = legp.charges[_p]*legp.qconj
            pp = 1 - 2*(cp[0]%2)
            qp = legp.get_qindex_of_charges(cp)
            sp = legp.get_slice(_p)
            #print('pp', pp, 'cp', cp, 'qp', qp, 'sp', sp)
            val = pp & pv
            for ip in range(sp.start, sp.stop):
                for iv in range(sv.start, sv.stop):
                    fswap[iv,ip,iv,ip] = val
    return fswap

def combine_three_copies(mps):
    """
    combining 3 copies with fermionic swap gates
    
    Inputs:
        1. mps, tempy.MPS object
        
    Outputs:
        1. mps3_Bslist, list of npc.Array
    """
    lx = mps.L
    mps3_Bslist = []
    
    for i in range(lx):
        tx = deepcopy(mps.get_B(i))
        ty = deepcopy(mps.get_B(i))
        tz = deepcopy(mps.get_B(i))
        
        #combining x and y
        legv = tx.get_leg('vR')
        tx.ireplace_label('vL','v1L')
        legp = ty.get_leg('p')
        fswap = fermion_swap_gate_z2(legv, legp)
        temp = npc.tensordot(tx, fswap, axes=(['vR'], ['fvL']))
        res = npc.tensordot(temp, ty, axes=(['sp*'], ['p']))
        
        res = res.combine_legs([['v1L','vL'], ['fvR','vR']], qconj=[+1, -1])
        res.ireplace_labels(['(v1L.vL)', '(fvR.vR)'], ['vL', 'vR'])
        res.legs[res.get_leg_index('vL')] = res.get_leg('vL').to_LegCharge()
        res.legs[res.get_leg_index('vR')] = res.get_leg('vR').to_LegCharge()
        res.itranspose(['vL', 'p', 'sp', 'vR'])
        
        #combining xy and z
        legv = res.get_leg('vR')
        res.ireplace_label('vL', 'v1L')
        legp = tz.get_leg('p')
        fswap = fermion_swap_gate_z2(legv, legp)
        fswap.ireplace_label('sp', 'ssp')
        temp = npc.tensordot(res, fswap, axes=(['vR'], ['fvL']))
        res = npc.tensordot(temp, tz, axes=(['sp*'], ['p']))
        
        res = res.combine_legs([['v1L','vL'], ['fvR','vR']], qconj=[+1, -1])
        res.ireplace_labels(['(v1L.vL)', '(fvR.vR)'], ['vL', 'vR'])
        res.legs[res.get_leg_index('vL')] = res.get_leg('vL').to_LegCharge()
        res.legs[res.get_leg_index('vR')] = res.get_leg('vR').to_LegCharge()
        res.itranspose(['vL', 'p', 'sp', 'ssp', 'vR'])
        mps3_Bslist.append(res)
    
    return mps3_Bslist


def gutzwiller_projection_npcarraylist(npcarraylist):
    """
    Gutzwiller projection function: 
    
               p
         ______|______
         |___________|           projector
           |   |   |
          p*1 p*2 p*3
          (contract)
           p   sp ssp
         __|___|___|__
     vL--|___________|--vR       npcarraylist[i]
     
    contracting the p* and sp legs.  
    
    Input: 
        1. npcarraylist, list of npc.Array, the combined, unprojected xyz mps
    Output:
        1. gp_npcarraylist, list of npc.Array, the projected xyz mps, labeled as 'vL', 'p', 'vR'
    """
    gp_npcarraylist = []
    lx = len(npcarraylist)
    
    spin1_site = SO3Site(cons_S='U1')
    spin1_leg = spin1_site.leg
    parton_site = oneparton(cons_S='U1')
    parton_leg = parton_site.leg
    
    projector = npc.zeros([spin1_leg, parton_leg.conj(), parton_leg.conj(), parton_leg.conj()], labels=['p','p*1','p*2','p*3'], dtype=npcarraylist[0].dtype)
    projector[0,1,0,0] = 1
    projector[1,0,1,0] = 1
    projector[2,0,0,1] = 1
    
    for i in range(lx):
        res = npc.tensordot(projector, npcarraylist[i], axes=(['p*1', 'p*2', 'p*3'], ['p', 'sp', 'ssp']))
        gp_npcarraylist.append(res)
    return gp_npcarraylist

In [11]:
class threeparton(Site):
    def __init__(self, cons_N=None, cons_S=None):
        self.conserve = [cons_N, cons_S]
        self.cons_N = cons_N
        self.cons_S = cons_S

        flavors = ['x', 'y', 'z']
        
        if cons_N == None and cons_S == 'U1':
            chinfo = npc.ChargeInfo([1], ['S'])
            leg = npc.LegCharge.from_qflat(chinfo, [[0], [-1], [0], [1], [-1], [0], [1], [0]]) #change x->-1 y->0 z->1 here to fit the Cartan subalgebra U(1) GQN !!!!!!!!!!KEY
        else:
            print('No symmetry used in site threeparton')
            leg = npc.LegCharge.from_trivial(8)

        names = ['empty', 'x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']

        JW = np.diag([1,-1,-1,-1,1,1,1,-1])
        Fx = np.diag([1,-1,1,1,-1,-1,1,-1])
        Fy = np.diag([1,1,-1,1,-1,1,-1,-1])
        id8 = np.diag([1,1,1,1,1,1,1,1])
        
        axdag = np.zeros((8,8))
        axdag[1,0] = 1; axdag[4,2] = 1; axdag[5,3] = 1; axdag[7,6] = 1; 
        ax = axdag.T
        aydag = np.zeros((8,8))
        aydag[2,0] = 1; aydag[4,1] = 1; aydag[6,3] = 1; aydag[7,5] = 1; 
        ay = aydag.T
        azdag = np.zeros((8,8))
        azdag[3,0] = 1; azdag[5,1] = 1; azdag[6,2] = 1; azdag[7,4] = 1; 
        az = azdag.T

        cxdag = axdag
        cx = ax
        cydag = aydag @ Fx
        cy = Fx @ ay
        czdag = azdag @ Fy @ Fx
        cz = Fx @ Fy @ az

        ops = dict(JW=JW, Fx=Fx, Fy=Fy, id8=id8, 
                   cxdag = cxdag, cx = cx, cydag = cydag, cy = cy, czdag = czdag, cz = cz) #with the JW string embedded in c operators, there is no need to add a operators into ops dict. 
        
        Site.__init__(self, leg, names, **ops)

In [12]:
def oneparton_to_threeparton(npcarraylist):
    #rub three oneparton sites into one threeparton site, output a list of npc arrays
    lx = len(npcarraylist)
    
    threepartonsite = threeparton(cons_N=None, cons_S='U1')
    threepartonleg = threepartonsite.leg
    onepartonsite = oneparton(cons_N='Z2')
    onepartonleg = onepartonsite.leg

    projector = npc.zeros([threepartonleg, onepartonleg.conj(), onepartonleg.conj(), onepartonleg.conj()], labels=['p','p*1','p*2','p*3'], dtype=npcarraylist[0].dtype)

    projector[0,0,0,0] = 1
    projector[3,0,0,1] = 1
    projector[2,0,1,0] = 1
    projector[6,0,1,1] = 1
    projector[1,1,0,0] = 1
    projector[5,1,0,1] = 1
    projector[4,1,1,0] = 1
    projector[7,1,1,1] = 1

    threeparton_nparraylist = []
    for i in range(lx):
        res = npc.tensordot(projector, npcarraylist[i], axes=(['p*1', 'p*2', 'p*3'], ['p', 'sp', 'ssp']))
        threeparton_nparraylist.append(res)
    return threeparton_nparraylist

### Main function

Question: Can I really add $U(1)$ good quantum number into the Pfaffian method? 

In [13]:
#parameters
lx = 6
J = 1.0
K = 0.0
chi = 1.0
delta = 0.98
lamb = 1.78
D = 16
pbc = 1

if pbc == 1:
    bc = 'periodic'
elif pbc == 0:
    bc = 'open'
else:
    raise "pbc must be 1(periodic) or 0(open)"

In [14]:
print('----------Pfaffian Start---------')
singlekitaev = singlechain(chi, delta, lamb, lx, D, pbc)
singlekitaev.hamiltonian()
mpsflat = []
for i in range(1,lx+1):
    tensortemp = singlekitaev.A_nba(i)
    mpsflat.append(tensortemp)
    
bflat = deepcopy(mpsflat)
for i in range(lx):
    if i == 0:
        bflat[i] = np.reshape(bflat[i], (1,2,2))
        bflat[i] = np.transpose(bflat[i], (1,0,2))
    elif i == lx-1:
        bflat[i] = np.reshape(bflat[i], (2,2,1))
#now bflat is a list of B-tensor in the leg label ['p','vL','vR'], ready for building MPS object

sites = [oneparton(flavor='x',cons_S='U1')] * lx
pfaffian_mps = MPS.from_Bflat(sites, bflat)
pfaffian_mps.canonical_form()

# Bs_list = combine_three_copies(pfaffian_mps) #Bs_list is in 2*2*2 (3*one parton sites)
# Bs_3p_list = oneparton_to_threeparton(Bs_list) #Bs_3p_list is in one 8-dim leg (1*three parton sites)
Bs_list = combine_three_copies(pfaffian_mps)
gp_Bs_list = gutzwiller_projection_npcarraylist(Bs_list)
    
spin1sites = [SO3Site(cons_S='U1')] * lx
gp_pfaffian_mps = MPS.from_product_state(spin1sites, [0]*lx)
for i in range(lx):
    gp_pfaffian_mps._B[i] = gp_Bs_list[i]
gp_pfaffian_mps.canonical_form()

----------Pfaffian Start---------
t= -1.0 d= 0.98 mu= 1.78
the eig energies [ 0.98117232  0.98117232  2.64764046  2.64764046  3.64621734  3.64621734
 -0.98117232 -0.98117232 -2.64764046 -2.64764046 -3.64621734 -3.64621734]
the exact energy by em and trt -7.275030118633545
check the hamiltonian diagonalization: HM=ME, M=[[u,v*];[v,u*]]
******************BdG--->MPS at site 1******************
check the correlation matrix diagonalization: RM=ME, M=[[u,v*];[v,u*]]
segmet = 0 <----> 0, L_p:
check the correlation matrix diagonalization: RM=ME, M=[[u,v*];[v,u*]]
segmet = 0 <----> 1, L_p:
state_b of length = 1:
 [[]]
state_a of length = 2:
 [[], [0]]
******************BdG--->MPS at site 2******************
check the correlation matrix diagonalization: RM=ME, M=[[u,v*];[v,u*]]
segmet = 0 <----> 1, L_p:
check the correlation matrix diagonalization: RM=ME, M=[[u,v*];[v,u*]]
segmet = 0 <----> 2, L_p:
state_b of length = 2:
 [[], [0]]
state_a of length = 4:
 [[], [1], [0], [0, 1]]
*****************

ValueError: wrong sector with non-zero entries